<a href="https://colab.research.google.com/github/KaichengDING/Triple-Defense/blob/main/Varying_noise_pred_idl_project_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Install ART

In [33]:
!pip install adversarial-robustness-toolbox==1.4.3

# Import Dependencies

In [34]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np
import pandas as pd

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier

from art.estimators.classification import EnsembleClassifier
from typing import List, Optional, Union, TYPE_CHECKING
from art.estimators.classification.classifier import ClassifierNeuralNetwork
from scipy.special import softmax

import matplotlib.pyplot as plt
import time
import logging
import datetime
import random

cuda = torch.cuda.is_available()

device = torch.device("cuda" if cuda else "cpu")


# Model Definition
`ShuffleNet` and `ShuffleNetV2` are actually identical, but models with seed number smaller than 200 are created using ShuffleNet and models with seed number greater than 200 are created using ShuffleNetV2. In order to load all models into ensemble, the two definitions are provided here.

In [35]:
class ShuffleNet(nn.Module):
    def __init__(self, nb_classes =10):
        super(ShuffleNet, self).__init__()
        self.shuffle = models.shufflenet_v2_x2_0()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.shuffle(x)
        x = self.linear(x)
        return x


class ShuffleNetV2(nn.Module):
    def __init__(self, nb_classes=10):
        super(ShuffleNetV2, self).__init__()
        self.shufflenet = models.shufflenet_v2_x2_0()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.shufflenet(x)
        x = self.linear(x)
        return x

# Logging Configuration

In [36]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
# logging on colab
file_hdlr = logging.FileHandler('/content/gdrive/My Drive/IDL_Project/logs/Rlog_{}.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

# Get Test Examples

In [37]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, X, Y, transform=None):
    self.X = X
    self.Y = Y
    self.transform = transform
  
  def __len__(self):
    return len(self.Y)
  
  def __getitem__(self, idx):
    if self.transform is None:
      return torch.from_numpy(self.X[idx]), torch.tensor(self.Y[idx]).long()
    else:
      return self.transform(self.X[idx]), torch.tensor(self.Y[idx]).long()

In [38]:
test_batchsize = 200
num_workers = 4
nb_classes = 10
img_size = 224
subset_size = 1000 #### CHANGED SUBSET SIZE

test_transform = transforms.Compose([transforms.ToPILImage(),
                                     transforms.Resize(size=img_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0, 0, 0), (1, 1, 1))])

testset= torchvision.datasets.CIFAR10(root='./data', train=False, download=True)
testset_data = testset.data[0:subset_size]
testset_labels = testset.targets[0:subset_size]

testset_sub = MyDataset(testset_data, testset_labels, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset_sub, batch_size=test_batchsize, shuffle=False, num_workers=num_workers, drop_last=True)

Files already downloaded and verified


# MyEnsembleClassifier
Inherited from `EnsembleClassifier` with `predict`, `loss_gradient` and `loss_gradient_framework` overwritten



In [53]:
class MyEnsembleClassifier(EnsembleClassifier):
    def __init__(
        self,
        classifiers: List[ClassifierNeuralNetwork],
        device,
        infer_noise,
        num_selected_models,
        n_prediction,
        n_loss,
        classifier_weights: Union[list, np.ndarray, None] = None,
        channels_first: bool = False,
        clip_values: Optional["CLIP_VALUES_TYPE"] = None,
        preprocessing_defences: Union["Preprocessor", List["Preprocessor"], None] = None,
        postprocessing_defences: Union["Postprocessor", List["Postprocessor"], None] = None,
        preprocessing: "PREPROCESSING_TYPE" = (0, 1),
    ) -> None:
      super().__init__(
          classifiers=classifiers,
          classifier_weights=classifier_weights,
          channels_first=channels_first,
          clip_values=clip_values,
          preprocessing_defences=preprocessing_defences,
          postprocessing_defences=postprocessing_defences,
          preprocessing=preprocessing
      )
      self.device = device
      self.infer_noise = infer_noise
      self.num_models = len(classifiers)
      self.num_selected_models = num_selected_models
      self.n_prediction = n_prediction
      self.n_loss = n_loss

    def predict(self, x: np.ndarray, batch_size: int = 128, raw: bool = False, **kwargs) -> np.ndarray:
        """
        Perform prediction for a batch of inputs. Predictions from classifiers should only be aggregated if they all
        have the same type of output (e.g., probabilities). Otherwise, use `raw=True` to get predictions from all
        models without aggregation. The same option should be used for logits output, as logits are not comparable
        between models and should not be aggregated.
        :param x: Test set.
        :param batch_size: Size of batches.
        :param raw: Return the individual classifier raw outputs (not aggregated).
        :return: Array of predictions of shape `(nb_inputs, nb_classes)`, or of shape
                 `(nb_classifiers, nb_inputs, nb_classes)` if `raw=True`.
        """
        indices = np.random.choice(self.num_models, self.n_prediction, replace=True)

        noise_list = [np.random.randn(*x.shape) * self.infer_noise for i in indices]

        preds = []
        for iidx, i in enumerate(indices):
          preds.append(softmax(self._classifiers[i].predict(np.float32(x + noise_list[iidx])), axis=1))
        preds = np.array(preds)


        del indices
        del noise_list
        torch.cuda.empty_cache()

        if raw:
            return preds

        # 6 x 100
        preds_classes = np.argmax(preds, axis=2)
        row, col = preds_classes.shape

        # 100,
        majority_vote = np.array(
            [
             np.bincount(preds_classes[:,c]).argmax()
             for c in range(col)
            ]
        )

        mask = preds_classes == majority_vote
        del majority_vote

        mask = np.repeat(np.expand_dims(mask, axis=2), repeats=10, axis=2)

        # Aggregate predictions only at probabilities level, as logits are not comparable between models
        var_z = np.sum(mask * preds, axis=0) / np.sum(mask, axis=0) # take mean (check sum to 1)
        del mask
        del preds

        # Apply postprocessing
        predictions = self._apply_postprocessing(preds=var_z, fit=False)
        del var_z

        return predictions


    def loss_gradient(self, x: np.ndarray, y: np.ndarray, raw: bool = False, **kwargs) -> np.ndarray:
        """
        Compute the gradient of the loss function w.r.t. `x`.
        :param x: Sample input with shape as expected by the model.
        :param y: Target values (class labels) one-hot-encoded of shape (nb_samples, nb_classes) or indices of shape
                  (nb_samples,).
        :param raw: Return the individual classifier raw outputs (not aggregated).
        :return: Array of gradients of the same shape as `x`. If `raw=True`, shape becomes `[nb_classifiers, x.shape]`.
        """

        indices = np.random.choice(self.num_models, self.num_selected_models, replace=True)

        noise_list = [np.random.randn(*x.shape) * self.infer_noise for i in indices]
        
        grads = np.array(
            [
                self._classifiers[i].loss_gradient(np.float32(x + noise_list[iidx]), y)
                for iidx, i in enumerate(indices)
            ]
        )

        torch.cuda.empty_cache()
        del indices
        del noise_list

        if raw:
            return grads

        return np.sum(grads, axis=0)

    def loss_gradient_framework(self, x: "torch.Tensor", y: "torch.Tensor", **kwargs) -> "torch.Tensor":
        """
        Compute the gradient of the loss function w.r.t. `x`.
        :param x: Sample input with shape as expected by the model.
        :param y: Target values (class labels) one-hot-encoded of shape (nb_samples, nb_classes) or indices of shape
                  (nb_samples,).
        :param raw: Return the individual classifier raw outputs (not aggregated).
        :return: Array of gradients of the same shape as `x`. If `raw=True`, shape becomes `[nb_classifiers, x.shape]`.
        """

        indices = np.random.choice(self.num_models, self.n_loss, replace=True)

        noise_list = [(torch.randn(x.size()) * self.infer_noise).to(device) for i in indices]


        # gradient shape: batch_size, 3, 224, 224
        accumulator = torch.zeros(test_batchsize, 3, img_size, img_size)
        for iidx, i in enumerate(indices):
          accumulator += self._classifiers[i].loss_gradient_framework(x + noise_list[iidx], y).cpu()
        
        torch.cuda.empty_cache()
        del indices
        del noise_list

        return accumulator.to(device)

# Experiment Settings
### common settings:
*   inference noise: 0.1
*   eps: 0.5
*   eps step: 0.4
*   max iter: 20
*   L2 norm
*   number of samplings: 50

### to experiment:
1. Total number of candidate models in the ensemble: 1 (randomized smoothing)
2. Total number of candidate models in the ensemble: 5
3. Total number of candidate models in the ensemble: 9
4. Total number of candidate models in the ensemble: 13

# Run this code block to load common variables

In [54]:
infer_noise = 0.1
eps = 0.5
eps_step = 0.4
max_iter = 20
norm = 2

# Load 5 models all

In [55]:
model_names = ['ShuffleNet_1',
               'ShuffleNet_2',
               'ShuffleNet_3',
               'ShuffleNet_4',
               'ShuffleNet_5']

Kaichen



In [56]:
n_prediction=5
n_loss=10,

Shivani

In [57]:
n_prediction=10
n_loss=5

# Kick off Experiment

# **The rest parts of the notebook was regarding previous work. Discard them for now.**

In [58]:
# load models
model_dict = {}
for model_name in model_names:
  model = ShuffleNet()
  # MAKE CHANGE
  model_data = torch.load('/content/gdrive/MyDrive/ShuffleNetModels/{}'.format(model_name), map_location=torch.device('cpu'))
  model.load_state_dict(model_data['model_state_dict'])
  model = model.to(device)
  model_dict[model_name] = model

logging.info("Models loaded successfully!")
logging.info("Total number of models: {}".format(len(model_names)))

# create classifier list
criterion = nn.CrossEntropyLoss()
classifier_list = []
for model_name in model_dict:
  classifier_list.append(PyTorchClassifier(model=model_dict[model_name],
                                           clip_values=(0, 1),
                                           loss=criterion,
                                           input_shape=(3, img_size, img_size),
                                           nb_classes=nb_classes))

2020-12-13 21:18:26,460 INFO Models loaded successfully!
2020-12-13 21:18:26,462 INFO Total number of models: 5
2020-12-13 21:18:26,464 INFO Inferred 1 hidden layers on PyTorch classifier.
2020-12-13 21:18:26,471 INFO Inferred 1 hidden layers on PyTorch classifier.
2020-12-13 21:18:26,476 INFO Inferred 1 hidden layers on PyTorch classifier.
2020-12-13 21:18:26,481 INFO Inferred 1 hidden layers on PyTorch classifier.
2020-12-13 21:18:26,487 INFO Inferred 1 hidden layers on PyTorch classifier.


In [ ]:
# create ensemble classifier
my_ensemble_classifier = MyEnsembleClassifier(classifier_list,
                                              device, 
                                              infer_noise=infer_noise,
                                              num_selected_models=1,
                                              n_prediction=n_prediction,
                                              n_loss=n_loss,
                                              clip_values=[0., 1.], 
                                              channels_first=True)

# create attack object
attack = ProjectedGradientDescentPyTorch(my_ensemble_classifier, 
                                         eps=eps, 
                                         eps_step=eps_step, 
                                         norm=norm, 
                                         max_iter=max_iter, 
                                         batch_size=test_batchsize)

# kick off experiment
Acc_adv = []
Acc_nat = []
for batch_idx, (X, Y) in enumerate(testloader):
  x_test = X.numpy()
  y_test = Y.numpy()
  predictions_nat = my_ensemble_classifier.predict(x_test)
  accuracy_nat = np.sum(np.argmax(predictions_nat, axis=1) == y_test) / len(y_test)
  Acc_nat.append(accuracy_nat)

  x_test_adv = attack.generate(torch.from_numpy(x_test), torch.from_numpy(y_test))
  predictions_adv = my_ensemble_classifier.predict(x_test_adv)
  accuracy_adv = np.sum(np.argmax(predictions_adv, axis=1) == y_test) / len(y_test)
  Acc_adv.append(accuracy_adv)

  logging.info('accuracy (adversarial): {}'.format(accuracy_adv))
  logging.info('accuracy (natural): {}'.format(accuracy_nat))
    

# saving the dataframe 
Acc_adv = np.asanyarray(Acc_adv)
Acc_nat = np.asanyarray(Acc_nat)
res = {'AccADV': Acc_adv, 'AccNAT': Acc_nat} 
df = pd.DataFrame(res)

#### MAKE CHANGE
df.to_csv('./gdrive/My Drive/IDL_Project/results/result_raphaelC_1avg_20{}models_{}samples.csv'.format(len(model_names), num_selected_models),index=True)